In [96]:
#import necessary libraries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

#ask user for product type can be any NYMEX contract
product = input('Which product contract settle prices do you need? eg. NG, CL, QA ARGUS and PLATTS contracts no longer avail :  ')
product = product.upper()

years = input('How many years worth of data do you want? (Max 10) : ')
years = int(years)

#set user type headers for request
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

#create list to store historical data
historical =[]

#months list needs to be initialized manually for current year contracts that have settled to date
#this can be improved to be automatically created, being lazy
months = ['G22', 'F22']

#month codes used across all NYMEX controcts (Z - December, X - November, etc.)
codes = ['Z', 'X', 'V', 'U', 'Q', 'N', 'M', 'K', 'J', 'H', 'G', 'F']

#get last two digits from last year for the following loop
currentTimeDate = datetime.now() - relativedelta(years=1)
currentTime = int(currentTimeDate.strftime('%y'))

#this loop appends to the months list for the number of years input by the user
for j in range (0,years):
    for i in range(len(codes)):
        year = currentTime - j
        months.append(codes[i]+str(year))

#this function grabs the url and data needed from the web page; requires two parameters
def getData(product, month):
    url = f'https://www.barchart.com/futures/quotes/{product}{month}' #define url to scrape with a page tag
    r = requests.get(url, headers=headers) #get the url
    soup = BeautifulSoup(r.text, 'html.parser') #read the url page and create a soup object
    data = soup.find('div', {'class': 'page-title symbol-header-info'})['data-ng-init']
    data = data[data.find('{'):data.rfind('}')+1]
    data = json.loads(data)
    settlement = {'CONTRACT' : data['symbol'], 'SETTLE' : float(data['lastPrice'].strip('s')), 'SETTLE DATE' : datetime.strptime(data['tradeTime'],'%m/%d/%y')}
    historical.append(settlement)
    return historical

#this loop iterates over all the contracts that are elements in the months list created earlier
for x in months: 
    getData(product,x)

#creates a dataframe with the historical settle prices   
df = pd.DataFrame(historical)

#saves the file as an excel sheet in the current directory this script is being run from
df.to_excel(f'{product}pricing.xlsx',sheet_name=product)

Which product contract settle prices do you need? eg. NG, CL, QA ARGUS and PLATTS contracts no longer avail :  cl
How many years worth of data do you want? (Max 10) : 10
